To check the connectivity of phonon modes on volumes, fit phonon modes with volumes using polynomial functions up to 5, and fit equations of state. Similar to the QHA module in CRYSTAL17. Required files & input parameters:
* Harmonic phonon calculation output by CRYSTAL17  
* The name list of harmonic phonon outputs, stored with temperature range in 'HA_input.dat'  

By Spica. Vir., ICL, Mar 07 - 22. spica.h.zhou@gmail.com

In [ ]:
import math
import os
import numpy as np

In [ ]:
class Mode:
    """
    mode class - store important information for mode with the same rank.
        rank, int, the rank of the mode object
        npoint, int, number of harmonic frequency calculations
        freq, numpy array, npoint * 1 array of frequencies corresponding to the same mode
        vol, numpy array, npoint * 1 array of volumes of harmonic input
        eigvt, numpy array, npoint * 3 array of corresponding eigenvector
    """

    def __init__(self, rank=0, npoint=0, freq=[], vol=[],  eigvt=[]):
        self.rank = rank
        self.npoint = npoint
        # Unit: THz
        self.freq = freq * 2 * math.pi
        # Unit: A^3
        self.vol = vol
        # Unit: A
        self.eigvt = eigvt

    def continuity(self, wtout, threshold=0.4):
        wtout.write('%-15s%6i\n' % ('FREQUENCY #', self.rank))
        wtout.write('%-15s%15s\n' % ('  VOL (A^3)', 'FREQ (cm^-1)'))
        for i in range(self.npoint):
            wtout.write('%2s%-15.4f%15.4f\n' % ('', self.vol[i], self.freq[i]))
            for j in range(i + 1, self.npoint):
                dot_eigvt = np.dot(self.eigvt[i], self.eigvt[j])
                if dot_eigvt < threshold:
                    wtout.write('%s%i%s\n'% ('Warning: Continuity between the above mode \
                    and mode ', j + 1, ' is problematic.'))
        
        wtout.write('\n')
        
        return self
        
    def poly_fit(self, wtout, power=[2, 3, 4]):
        if max(power) > self.npoint - 1:
            wtout.write('%s\n' % 'Error: Lack referece data for polynomial fit.')
            
            return
        
        for i in power:
            func = np.polynomial.polynomial.Polynomial.fit(self.vol, self.freq, i)
            ss_res = np.sum((self.freq - func(self.vol))**2)
            ss_tot = np.sum((self.freq - np.mean(self.freq))**2)
            r_square = 1 - ss_res / ss_tot
            
    